<a href="https://colab.research.google.com/github/luisin-py/Conversor-imagens-para-.tex/blob/main/transforme_latex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
def transforme_latex(path_imagem):
  """
  Função que recebe o caminho de uma imagem e tenta convertê-la em código LaTeX através da API do Google Generative AI.
  """

  from pathlib import Path
  import hashlib
  import google.generativeai as genai

  # Configura a chave de API para acesso à API do Google Generative AI
  genai.configure(api_key="chaveAPI")

  # Configurações para geração do texto
  generation_config = {
      "temperature": 1,  # Define a criatividade do modelo (0 = menos criativo, 1 = mais criativo)
      "top_p": 0.95,    # Define a probabilidade cumulativa de tokens a serem considerados
      "top_k": 0,       # Define o número de tokens a serem considerados (0 = desativado)
      "max_output_tokens": 8192, # Define o número máximo de tokens na saída
  }

  # Configurações de segurança para evitar conteúdo impróprio
  safety_settings = [
      {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
      {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
      {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
      {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
  ]

  # Inicializa o modelo generativo com as configurações definidas
  model = genai.GenerativeModel(
      model_name="gemini-1.5-pro-latest",
      generation_config=generation_config,
      safety_settings=safety_settings
  )

  # Lista para armazenar arquivos carregados
  uploaded_files = []

  def upload_if_needed(pathname: str) -> list[str]:
    """
    Função auxiliar que verifica se o arquivo já foi carregado e,
    caso contrário, faz o upload e retorna a lista de arquivos carregados.
    """
    path = Path(pathname)
    hash_id = hashlib.sha256(path.read_bytes()).hexdigest() # Gera um hash único para o arquivo

    try:
      # Tenta obter o arquivo já carregado pelo seu hash
      existing_file = genai.get_file(name=hash_id)
      return [existing_file]
    except:
      pass

    # Se o arquivo não existe, faz o upload
    uploaded_files.append(genai.upload_file(path=path, display_name=hash_id))
    return [uploaded_files[-1]]

  # Cria o prompt para o modelo, incluindo o texto de instrução e o arquivo da imagem
  prompt_parts = [
      "Transforme essa imagem em arquivo .tex  \n\n",
      *upload_if_needed(str(path_imagem)),
      "\n",
  ]

  # Gera o conteúdo (código LaTeX) a partir do prompt
  response = model.generate_content(prompt_parts)

  # Imprime o código LaTeX gerado
  print(response.text)

  # Remove os arquivos carregados após a geração do código
  for uploaded_file in uploaded_files:
      genai.delete_file(name=uploaded_file.name)